<a href="https://www.kaggle.com/code/madeeltariq/api-gateway-rag?scriptVersionId=214137779" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# *Installing the required libraries*# 

In [ ]:
import subprocess
import sys
import os

def install_package(package_name):
    try:
        # Check if the package is installed
        subprocess.check_call([sys.executable, "-m", "pip", "show", package_name])
        print(f"{package_name} is already installed.")
    except subprocess.CalledProcessError:
        # If package is not installed, install it
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

def install_ngrok():
    # Check if ngrok is already downloaded and extracted
    if not os.path.exists('ngrok'):
        print("Downloading and installing ngrok...")
        subprocess.run(['wget', '-q', '-O', 'ngrok.zip', 'https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip'])
        subprocess.run(['unzip', '-o', 'ngrok.zip'])
        print("ngrok installed successfully.")
    else:
        print("ngrok is already installed.")

# Installing packages step-by-step
install_package("flask")
install_package("transformers")
install_package("pinecone-client")
install_package("python-dotenv")
install_package("flask-cors")
install_package("pyngrok")

# Install ngrok separately
install_ngrok()

print("Done With All installations >>>>>>>>>>>>>>>>> Move Forward")


# *Checking the .env variables***

In [ ]:
import requests

# Replace with the actual server address (e.g., ngrok URL or your local server)
url = "https://7c4a-34-57-216-57.ngrok-free.app/health"

# Send GET request to check if the server is up
try:
    response = requests.get(url)
    if response.status_code == 200:
        print("Flask server is running:", response.json())
    else:
        print("Flask server is not responding correctly.")
except requests.exceptions.RequestException as e:
    print("Error while connecting to Flask server:", e)


# Implemented Ngrok and CORS on top of the Flask API

## Code cell for Entering the ngrok API key For Ngrok implementation

In [ ]:
import os

# Function to create/update .env file with ngrok API key
def set_ngrok_api_key():
    # Ask user for the ngrok API key
    ngrok_api_key = input("Please enter your ngrok API key: ")
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the key; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    
    print(f"ngrok API key added to {env_file}")

# Call the function to set the API key
set_ngrok_api_key()


In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS to handle cross-origin requests
from pyngrok import ngrok  # Import ngrok
from dotenv import load_dotenv  # Import dotenv to load .env file

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Health check endpoint with request body and logging
@app.route('/health', methods=['GET', 'POST'])
def health_check():
    try:
        # Log the request data and ID (if present)
        if request.method == 'POST':
            data = request.json
            print("Received request data:", data)
            id_value = data.get("id", "No ID provided")
            print("Received ID:", id_value)
        else:
            print("Received request data:", request.args)
        
        # Respond with a Good message
        return jsonify({"status": "Flask server is running great!"})
    except Exception as e:
        # Catch any exceptions and respond with an error message
        print(f"Error: {str(e)}")
        return jsonify({"status": "Error occurred while processing the request."}), 500

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

# Function to run Flask server
def run_flask():
    print("Flask server is listening on port 5000...")
    app.run(host="0.0.0.0", port=5000)  # Running on port 5000

# Function to start ngrok and expose the Flask server
def run_ngrok():
    # Authenticate ngrok using the API key from the .env file
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
        print("Ngrok authenticated successfully.")
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    # Open a ngrok tunnel to the Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url

# Start Flask server in a separate thread to keep the notebook running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Start ngrok and print the URL where Flask is accessible
public_url = run_ngrok()

# Keep the notebook running to maintain the server
while True:
    time.sleep(1)  # Just keeps the notebook running


Flask server is listening on port 5000...
 * Serving Flask app '__main__'
 * Debug mode: off
Ngrok authenticated successfully.
Ngrok public URL: NgrokTunnel: "https://c89e-34-71-204-254.ngrok-free.app" -> "http://localhost:5000"
Received request data: {'id': 12345}
Received ID: 12345
Received request data: {'id': 12345, 'url': 'httpsasdfadsf'}
Received ID: 12345


# *Working Prototype for flask listening***

In [ ]:
import os
from flask import Flask, request, jsonify
import threading
import time

# Initialize Flask app
app = Flask(__name__)

# Health check endpoint
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "Flask server is running Greate!"})

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

# Function to run Flask server
def run_flask():
    print("Flask server is listening on port 8000...")
    app.run(host="0.0.0.0", port=9000)  # Running on port 8000

# Start Flask server in a separate thread to keep the notebook running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Print out the URL where Flask is running
print("Flask server should be accessible at: http://0.0.0.0:8000")

# Keep the notebook running to maintain the server
while True:
    time.sleep(1)  # Just keeps the notebook running
